<a href="https://colab.research.google.com/github/justdvnsh/Generator/blob/master/Char_level_lyric_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io
import re

Using TensorFlow backend.


In [0]:
import pandas as pd

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/rasbt/musicmood/master/dataset/training/train_lyrics_1000.csv')
df.head()

,file,artist,title,lyrics,genre,mood,year
0,TRAAAAW128F429D538.h5,Casual,I Didn't Mean To,Verse One:\n\nAlright I might\nHave had a litt...,Hip Hop/Rap,sad,1994
1,TRAAAEF128F4273421.h5,Adam Ant,Something Girls,Adam Ant/Marco Pirroni\nEvery girl is a someth...,Rock,happy,1982
2,TRAAAFD128F92F423A.h5,Gob,Face the Ashes,"I've just erased it's been a while, I've got a...",Rock,sad,2007
3,TRAABJV128F1460C49.h5,Lionel Richie,Tonight Will Be Alright,Little darling \nWhere you've been so long \nI...,R&B,happy,1986
4,TRAABLR128F423B7E3.h5,Blue Rodeo,Floating,"Lead Vocal by Greg\n\nWell, these late night c...",Rock,sad,1987


In [4]:
text = ''
# merge all the lyrics together into one huge string
for index, row in df['lyrics'].iteritems():
    text = text + str(row).lower()
    
# find all the unique chracters
chars = sorted(list(set(text)))
print('total chars:', len(chars))

# create a dictionary mapping chracter-to-index
char_indices = dict((c, i) for i, c in enumerate(chars))

# create a dictionary mapping index-to-chracter
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text into sequences
maxlen = 20
step = 1 # step size at every iteration
sentences = [] # list of sequences
next_chars = [] # list of next chracters that our model should predict

# iterate over text and save sequences into lists
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
# create empty matrices for input and output sets 
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

# iterate over the matrices and convert all characters to numbers
# basically Label Encoding process and One Hot vectorization
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 93


In [5]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               113664    
_________________________________________________________________
dense_1 (Dense)              (None, 93)                11997     
_________________________________________________________________
activation_1 (Activation)    (None, 93)                0         
Total params: 125,661
Trainable params: 125,661
Non-trainable params: 0
_________________________________________________________________


In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, logs):
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [8]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

history = model.fit(
    x, 
    y,
    batch_size=128,
    epochs=10,
    callbacks=[print_callback]
)

Epoch 1/10
1231455/1231455 [==============================] - 502s 407us/step - loss: 1.6724
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "be y'all (tribe y'al"
be y'all (tribe y'all get a same
the streets and it will see the searched with me
i want to be stranger

i want to be the stranger

i'm the finally the strangers
the start that i can't see the sensation of the strangers
and i want to be the stranger

i still see the side
i want to be the sear
and i want to be the stranger

i can't be the stranger

i want to see you and the strangers and the start
and i can't the seat
----- diversity: 0.5
----- Generating with seed: "be y'all (tribe y'al"
be y'all (tribe y'all darlin' to see it to be the need in the hand

and i'm everything out i gotta get and shake it out
i'll get all the knowned in the offown on the seens and the feel
the babess will see the rotchest broken show workin all the fores and that you won't see you acting and the back, can't 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


ay it's the death and you search and lonely
no things can't little decheath up and they say i was not my mind
there thing i can't be the body we be the singer out of the money
we're the world of the girl is this
i'm trying to see you

i should about to the heart of the singrion
i know it with a way that is when i 
Epoch 7/10
1231455/1231455 [==============================] - 601s 488us/step - loss: 1.5552
----- Generating text after Epoch: 6
----- diversity: 0.2
----- Generating with seed: "or disaster i will c"
or disaster i will come and the something in the same to the start
i got a singing on the stare
the stared is all the street
and i was the sings and the sick and so the sing of the same to see the stare and the street in the street
and the same is some in the fall
i was the street that so the same to come and like the same to the stare to the start
the booking on my side
i say it to be still (hear me so long book and
----- diversity: 0.5
----- Generating with seed: "or disaster

In [0]:
def generate_output():
    generated = ''
    usr_input = input("Write the beginning of your poem, the machine will complete it. Your input is: ")

    sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    generated += usr_input 

    sys.stdout.write("\n\nHere is your poem: \n\n") 
    sys.stdout.write(usr_input)
    for i in range(400):

        x_pred = np.zeros((1, Tx, len(chars)))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature = 0.2)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

        if next_char == '\n':
            continue

In [12]:
Tx = 20
generate_output()

Write the beginning of your poem, the machine will complete it. Your input is: rolling


Here is your poem: 

rollinge and 
i want to want to have the stre

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


at

free

i'm gonna see the streat

i want to be the of a will want to be the fuck of the to the of the life all to the more the thing to the love the& and the life the face we start thejre
i thing got th& be the & she's a sunge and i that i got the stuy the stars the stard the thing to the love the& when the streess with the & baby the on the rolled to me and